In [1]:
import numpy as np
from load import load_data
from convert import convert
from pmc import *

In [2]:
#%matplotlib widget
#%matplotlib inline
%matplotlib qt
#%matplotlib gtk

import matplotlib.pyplot as plt
import matplotlib.font_manager

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.rcParams.update({
    'pgf.rcfonts': False,
})

plt.rcParams['font.family'] = 'serif'

from matplotlib.backends.backend_pgf import FigureCanvasPgf
matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

In [3]:
bpC = 400 #CT Compression data Break Point in p-q plane
bpE = 150 #CT Extension data Break Point in p-q plane 

In [4]:
data = load_data(9)
d = convert(data)
P1, P2 = create_P1_and_P2(d,bpC,bpE)

in this situation $\phi_C^1 > \phi_C^2$ in this situation. planes P1 and P2 in zone 0, 2 and 4 do not intersect

if $\phi_E^1 > \phi_E^2$ planes P1 and P2 in zone 1, 3 and 5 do not intersect

In [5]:
print('P1 P2\n{} {}\n{} {}'.format(P1['phyC']*180/pi,P2['phyC']*180/pi,P1['phyE']*180/pi,P2['phyE']*180/pi))

P1 P2
31.463314872369153 29.890501808856005
27.819624911617833 41.69821864862035


In [6]:
special_case_phyC = P1['phyC'] > P2['phyC']
special_case_phyE = P1['phyE'] > P2['phyE']

In [7]:
print('P1 < P2 ?\nphyC : {}\nphyE : {}'.format(special_case_phyC,special_case_phyE))

P1 < P2 ?
phyC : True
phyE : False


## Calculating intersections

### P1 P2 transition points

In [8]:
eq_pts_transisiton_P12 = np.zeros((6,3,3))

for i in range(6):
    eq_pts_transisiton_P12[i] = np.array([get_plane_n(P1,i),get_plane_n(P1,i-1),get_plane_n(P2,i)])
    
ones = np.ones((6,3))

pts_transisiton_P12 = np.linalg.solve(eq_pts_transisiton_P12, ones).transpose()

In [9]:
pts_transisiton_P12 = pts_transisiton_P12[:,1::2]
print(pts_transisiton_P12)

[[ 18.3606332   18.3606332  -32.14509724]
 [-32.14509724  18.3606332   18.3606332 ]
 [ 18.3606332  -32.14509724  18.3606332 ]]


$V_0^2$

In [10]:
eq_pts_P2 = np.array([get_plane_n(P2,0),get_plane_n(P2,1),get_plane_n(P2,2)])
pts_P2 = np.linalg.solve(eq_pts_P2,np.ones((3,1)))

### Intersection with far plane

In [11]:
def twelve_plane_cycle(P1,P2,num,offset=0):
    num = (num + offset)%12
    p_cycle = [P2,P1,P1,P2]
    P = p_cycle[num%4]
    return get_plane_n(P,num//2)

In [12]:
print(twelve_plane_cycle(P1,P2,3,2))
print(twelve_plane_cycle(P1,P2,5,0))

[ 0.00750689  0.00185949 -0.02575906]
[ 0.00750689  0.00185949 -0.02575906]


In [13]:
sig_pts = np.array([d['sig2'],d['sig3'],d['sig1']])
all_pts = np.concatenate((sig_pts,pts_transisiton_P12,pts_P2),axis=1) #miss pts_P2
pts_with_largest_coord = 1*all_pts[:,np.argmax(np.max(all_pts, axis=0))]

far_plane_n = np.ones(3)/np.sqrt(3)
far_plane_o = far_plane_n*(np.dot(far_plane_n,pts_with_largest_coord)*1.5)

In [14]:
eq_pts_far = np.zeros((12,3,3))

b_far = np.array([[1,1,np.dot(far_plane_n,far_plane_o)]])

for i in range(12):
    eq_pts_far[i] = np.array([twelve_plane_cycle(P1,P2,i,'phyC'),twelve_plane_cycle(P1,P2,i-1,'phyC'),far_plane_n])
    
pts_far = np.linalg.solve(eq_pts_far,b_far).transpose()

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
print(pts_far)

## plotting 3D

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.set_aspect('equal')
ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')

In [ ]:
ax.plot(*pts_transisiton_P12[:,:2], 'ro',alpha=1)
ax.plot(*pts_transisiton_P12[:,-1:], 'yo',alpha=1)
ax.plot(*pts_P2,'ko',alpha=1)
ax.plot(*pts_far,'bo',alpha=1)

In [ ]:
pts_transisiton_P12[:,-1:]

In [ ]:
def get_verts(cord):
    return [list(zip(*cord))]

## Draw P2 planes

In [ ]:
for i in range(3):
    cord_l = np.zeros((3,4))
    cord_r = np.zeros((3,4))
    far_pts_inter_P2_idx = i*4
    cord_r[:,0] = pts_P2[:,0]
    cord_r[:,1] = pts_transisiton_P12[:,i] 
    cord_r[:,2] = pts_far[:,(far_pts_inter_P2_idx+1)%12]
    cord_r[:,3] = pts_far[:,far_pts_inter_P2_idx]
    
    cord_l[:,0] = pts_P2[:,0]
    cord_l[:,1] = pts_transisiton_P12[:,(i-1)%3] 
    cord_l[:,2] = pts_far[:,(far_pts_inter_P2_idx-1)%12]
    cord_l[:,3] = pts_far[:,far_pts_inter_P2_idx]
    
    verts = get_verts(cord_r)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)
    
    verts = get_verts(cord_l)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)

## Draw P1 planes

In [ ]:
for i in range(3):
    cord_l = np.zeros((3,3))
    cord_r = np.zeros((3,3))
    far_pts_inter_P2_idx = i*4 + 2
    cord_r[:,0] = pts_transisiton_P12[:,i] 
    cord_r[:,1] = pts_far[:,(far_pts_inter_P2_idx+1)%12]
    cord_r[:,2] = pts_far[:,far_pts_inter_P2_idx]

    cord_l[:,0] = pts_transisiton_P12[:,i] 
    cord_l[:,1] = pts_far[:,(far_pts_inter_P2_idx-1)%12]
    cord_l[:,2] = pts_far[:,far_pts_inter_P2_idx]
    
    verts = get_verts(cord_r)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)
    
    verts = get_verts(cord_l)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)

In [ ]:
ax.set_aspect('equal')
fig.show()

In [ ]:
-1 // 2

In [ ]:
d

In [ ]:
P1['Vo']

In [ ]:
P2['Vo']